In [ ]:
# Instalar la biblioteca de Gemini (si no está ya instalada)
!pip install google-genai pandas

import pandas as pd
import os
import json
from google.colab import drive
# Se importa el cliente principal y los tipos de respuesta
from google.genai import Client, types

# --- IMPORTACIÓN Y CONFIGURACIÓN CORREGIDA ---
# Pega tu NUEVA clave de API aquí
API_KEY = "XXXXXX"

# 1. Crear la instancia del Cliente, pasando la clave de API directamente.
gemini_client = Client(api_key=API_KEY)

# 2. Definir el nombre del modelo como una cadena.
# ¡IMPORTANTE! Eliminamos la llamada problemática 'models.get()' para evitar el error.
model_name = 'gemini-2.5-flash'


# Montar Google Drive
drive.mount('/content/drive')

# Ruta a la carpeta que contiene tus archivos .txt
transcripciones_path = "/content/drive/MyDrive/TFG_2025/Entrevistas/"

def analizar_entrevista_con_gemini(transcripcion_texto, nombre_entrevista):
    """
    Usa la IA para analizar la transcripción y extraer datos estructurados.
    """
    prompt = f"""
    Eres un analista de datos de investigación cualitativa para un TFG sobre el impacto de las Artes Electrónicas, la Psicofisiología y la Metacognición en la recuperación del estrés.

    Analiza la siguiente transcripción. Tu tarea es extraer la información clave y devolverla en el formato JSON especificado.

    TRANSCRIPCIÓN: "{transcripcion_texto[:4000]}..."

    ---

    Instrucciones para el análisis:
    1.  SENTIMIENTO_GENERAL: Evalúa el sentimiento dominante (Positivo, Negativo, o Neutral/Mixto).
    2.  METACONCIENCIA_CLAVE: Extrae 1-2 frases del participante donde reflexione sobre su propio estado o proceso de pensamiento ("Me di cuenta de que...", "Sentí que...", "Pienso que...").
    3.  ESTRES_CALMA_CLAVE: Extrae 1-2 frases del participante que describan claramente su experiencia de estrés o calma/recuperación.
    4.  HALLAZGOS_RESUMEN: Escribe un resumen de 2-3 oraciones sobre el hallazgo más relevante de esta entrevista para el TFG (e.g., "El participante asoció directamente la música con una caída en el estrés percibido.").

    Devuelve la respuesta ÚNICAMENTE en formato JSON, sin texto explicativo.
    """

    try:
        # Se pasa el nombre del modelo como cadena de texto (model_name)
        response = gemini_client.models.generate_content(
            model=model_name,
            contents=prompt,
            config=types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema={
                    "type": "object",
                    "properties": {
                        "entrevista_id": {"type": "string"},
                        "sentimiento_general": {"type": "string", "description": "Positivo, Negativo, o Neutral/Mixto."},
                        "metaconciencia_clave": {"type": "string"},
                        "estres_calma_clave": {"type": "string"},
                        "hallazgos_resumen": {"type": "string"}
                    },
                    "required": ["entrevista_id", "sentimiento_general", "metaconciencia_clave", "estres_calma_clave", "hallazgos_resumen"]
                }
            )
        )
        # El contenido es una cadena JSON, lo convertimos a diccionario
        return json.loads(response.text)

    except Exception as e:
        print(f"Error al analizar {nombre_entrevista}: {e}")
        return None

# Lista para guardar los resultados estructurados
resultados_analisis = []

# Iterar sobre los archivos y aplicar la función de análisis
for filename in os.listdir(transcripciones_path):
    if filename.endswith(".txt"):
        ruta_archivo = os.path.join(transcripciones_path, filename)

        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            transcripcion_content = f.read()

        print(f"Analizando: {filename}")

        # Le damos un ID simple a la entrevista
        entrevista_id = filename.split('.')[0]

        # Llamamos a la función de análisis
        resultado = analizar_entrevista_con_gemini(transcripcion_content, entrevista_id)

        if resultado:
            # Aseguramos que el ID de la entrevista esté en el JSON final
            resultado['entrevista_id'] = entrevista_id
            resultados_analisis.append(resultado)

# 4. Convertir los resultados a un DataFrame de pandas
df_resultados_final = pd.DataFrame(resultados_analisis)

print("\n--- ANÁLISIS AUTOMATIZADO COMPLETADO ---")
print(df_resultados_final)

# 5. Guardar el resultado final en tu Drive
output_path = "/content/drive/MyDrive/TFG_2025/Analisis_Automatico_TFG.csv"
df_resultados_final.to_csv(output_path, index=False)
print(f"\nResultados guardados en Drive: {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Analizando: ale2311092025.txt
Analizando: rue1022092025.txt
Analizando: lli2108092025.txt
Analizando: ben2004092025.txt
Analizando: any0828082025.txt
Analizando: vas2804092025.txt
Analizando: yes1904092025.txt
Analizando: asi1621082025.txt
Analizando: bio0921082025.txt
Analizando: ESI0821082025.txt
Analizando: uro1628082025.txt
Analizando: osa1111092025.txt
Analizando: nto0828082025.txt
Analizando: eva1308092025.txt
Analizando: mez1402102025.txt
Analizando: rez1122092025.txt
Analizando: ino0821082025.txt

--- ANÁLISIS AUTOMATIZADO COMPLETADO ---
    entrevista_id sentimiento_general  \
0   ale2311092025            Positivo   
1   rue1022092025            Positivo   
2   lli2108092025       Neutral/Mixto   
3   ben2004092025            Positivo   
4   any0828082025       Neutral/Mixto   
5   vas2804092025       Neutral/Mixto   
6   yes1904092025            Pos